This is a test just to make sure the StructuralLSTM 1D model works without anything breaking -- not expecting any actual useful performance metrics.

In [1]:
# What we're here for
from structural_lstm_1d import StructuralLSTM

# Everything else
from keras.layers import Dense, Embedding, LSTM
from keras.models import Sequential

# Preprocessing
from keras.preprocessing import sequence
#from nltk.corpus import stopwords

# Data
from keras.datasets import imdb

Using TensorFlow backend.


In [2]:
# Training data and input parameters
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32
epochs = 5

In [3]:
# Build the test model
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(StructuralLSTM(  # What we're testing!!!
    units = 128,
    filters = [4, 8, 16],
    kernel_size = 4,
    conv_layers = 3,
    dropout = 0.2,
    recurrent_dropout = 0.2,
    structure_dropout = 0.2
))
model.add(Dense(1, activation='sigmoid'))

Build model...


TypeError: Expected binary or unicode string, got None

In [ ]:
# Build the reference LSTM model
print('Build reference model...')
model_base = Sequential()
model_base.add(Embedding(max_features, 128))
model_base.add(LSTM(  # What we're testing!!!
    units = 128,
    dropout = 0.2,
    recurrent_dropout = 0.2
))
model_base.add(Dense(1, activation='sigmoid'))

In [ ]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

# Preprocessing
print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen = maxlen)
x_test = sequence.pad_sequences(x_test, maxlen = maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train both to compare...')
for epoch in range(epochs):
    print('\tTraining StructuralLSTM...')
    model.fit(x_train, y_train,
              batch_size = batch_size,
              epochs = 1, initial_epoch = epoch,
              validation_data = (x_test, y_test),
              verbose = 1)  # verbose = 1 has a progress bar
    print('\tTraining basic LSTM...')
    model_base.fit(x_train, y_train,
              batch_size = batch_size,
              epochs = 1, initial_epoch = epoch,
              validation_data = (x_test, y_test),
              verbose = 1)  # verbose = 1 has a progress bar
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)
score_base, acc_base = model_base.evaluate(x_test, y_test, batch_size=batch_size)
print('Test scores:', score, ',', score_base)
print('Test accuracies:', acc, ',', acc_base)